# 深度池化：沿通道维度的池化操作

传统池化（MaxPool2D, AvgPool2D）在空间维度(H, W)上进行下采样。
**深度池化**则是沿通道维度(C)进行池化，用于：

1. **降低通道数** - 减少特征图的深度
2. **特征融合** - 将相邻通道的特征进行聚合
3. **学习通道间的不变性** - 类似于空间池化学习平移不变性

本教程涵盖：
- 深度最大池化的原理与实现
- 使用Lambda层封装为Keras层
- 全局平均池化的等效实现

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

## 第一部分：理解深度池化

### 1.1 深度池化 vs 空间池化

| 池化类型 | 操作维度 | 输入形状 | 输出形状 |
|---------|---------|---------|----------|
| 空间池化 (2x2) | H, W | (B, H, W, C) | (B, H/2, W/2, C) |
| 深度池化 (ksize=3) | C | (B, H, W, C) | (B, H, W, C/3) |

In [ ]:
# 加载示例图像
from sklearn.datasets import load_sample_image

# 加载RGB图像并归一化
china = load_sample_image("china.jpg") / 255.0
flower = load_sample_image("flower.jpg") / 255.0

# 构建批次数据
images = np.array([china, flower], dtype=np.float32)
batch_size, height, width, channels = images.shape

print(f"输入图像形状: {images.shape}")
print(f"批次大小: {batch_size}")
print(f"空间尺寸: {height}x{width}")
print(f"通道数: {channels} (RGB)")

### 1.2 使用tf.nn.max_pool实现深度池化

`tf.nn.max_pool`的`ksize`参数可以指定四个维度的池化窗口大小：
- `ksize = (batch, height, width, channels)`
- 深度池化设置: `ksize = (1, 1, 1, depth_factor)`

In [ ]:
# 深度最大池化实现
# ksize=(1,1,1,3) 表示在通道维度上每3个通道取最大值
# strides=(1,1,1,3) 表示通道维度步幅为3，窗口不重叠

# 注意：深度池化目前只能在CPU上运行
with tf.device('/CPU:0'):
    depth_pooled = tf.nn.max_pool(
        input=images,
        ksize=(1, 1, 1, 3),      # 在通道维度上池化窗口大小为3
        strides=(1, 1, 1, 3),    # 通道维度步幅为3
        padding='SAME'
    )

print(f"输入形状: {images.shape}")
print(f"深度池化后形状: {depth_pooled.shape}")
print(f"通道数从 {images.shape[-1]} 降为 {depth_pooled.shape[-1]}")

In [ ]:
# 理解深度池化的计算过程
# 对于RGB图像，每个像素有3个通道值
# 深度池化(ksize=3)会将R,G,B三个值取最大，输出单通道

# 手动验证
sample_pixel = images[0, 100, 200, :]  # 取一个像素的RGB值
pooled_pixel = depth_pooled[0, 100, 200, 0]  # 对应的池化结果

print(f"原始RGB值: R={sample_pixel[0]:.4f}, G={sample_pixel[1]:.4f}, B={sample_pixel[2]:.4f}")
print(f"max(R,G,B) = {np.max(sample_pixel):.4f}")
print(f"深度池化结果: {pooled_pixel:.4f}")
print(f"验证: {'通过' if np.abs(np.max(sample_pixel) - pooled_pixel) < 1e-6 else '失败'}")

In [ ]:
# 可视化深度池化效果
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 原始彩色图像
axes[0, 0].imshow(images[0])
axes[0, 0].set_title('原始RGB图像')
axes[0, 0].axis('off')

# 深度池化结果（单通道，显示为灰度）
axes[0, 1].imshow(depth_pooled[0, :, :, 0], cmap='gray')
axes[0, 1].set_title('深度池化结果\n(max across R,G,B)')
axes[0, 1].axis('off')

# 花朵图像
axes[1, 0].imshow(images[1])
axes[1, 0].set_title('原始RGB图像')
axes[1, 0].axis('off')

axes[1, 1].imshow(depth_pooled[1, :, :, 0], cmap='gray')
axes[1, 1].set_title('深度池化结果\n(max across R,G,B)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 第二部分：封装为Keras层

### 2.1 使用Lambda层

In [ ]:
def create_depth_max_pool_layer(depth_factor=3):
    """
    创建深度最大池化层
    
    Parameters
    ----------
    depth_factor : int
        通道维度的池化窗口大小，输出通道数 = 输入通道数 / depth_factor
    
    Returns
    -------
    keras.layers.Lambda
        可用于Sequential或Functional API的深度池化层
    """
    return keras.layers.Lambda(
        lambda X: tf.nn.max_pool(
            X,
            ksize=(1, 1, 1, depth_factor),
            strides=(1, 1, 1, depth_factor),
            padding='SAME'
        )
    )

# 测试
depth_pool_layer = create_depth_max_pool_layer(depth_factor=3)

with tf.device('/CPU:0'):
    layer_output = depth_pool_layer(images)
    
print(f"Lambda层深度池化输出形状: {layer_output.shape}")

### 2.2 自定义Layer类（更灵活的实现）

In [ ]:
class DepthMaxPooling2D(keras.layers.Layer):
    """
    深度最大池化层 - 沿通道维度进行最大池化
    
    与空间池化不同，此层在通道(深度)维度上进行下采样，
    可用于降低特征图的通道数同时保留最显著的特征。
    
    Parameters
    ----------
    pool_size : int
        沿通道维度的池化窗口大小
    strides : int, optional
        沿通道维度的步幅，默认等于pool_size
    padding : str
        填充方式，'SAME'或'VALID'
    """
    
    def __init__(self, pool_size=3, strides=None, padding='SAME', **kwargs):
        super().__init__(**kwargs)
        self.pool_size = pool_size
        self.strides = strides if strides is not None else pool_size
        self.padding = padding
    
    def call(self, inputs):
        return tf.nn.max_pool(
            inputs,
            ksize=(1, 1, 1, self.pool_size),
            strides=(1, 1, 1, self.strides),
            padding=self.padding
        )
    
    def compute_output_shape(self, input_shape):
        batch, h, w, c = input_shape
        if self.padding == 'SAME':
            new_c = int(np.ceil(c / self.strides))
        else:
            new_c = int(np.ceil((c - self.pool_size + 1) / self.strides))
        return (batch, h, w, new_c)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'pool_size': self.pool_size,
            'strides': self.strides,
            'padding': self.padding
        })
        return config

# 测试自定义层
custom_depth_pool = DepthMaxPooling2D(pool_size=3)

with tf.device('/CPU:0'):
    custom_output = custom_depth_pool(images)
    
print(f"自定义层深度池化输出形状: {custom_output.shape}")

## 第三部分：全局平均池化的等效实现

全局平均池化可以使用`tf.reduce_mean`实现

In [ ]:
# 模拟CNN特征图输出
feature_maps = np.random.randn(2, 7, 7, 256).astype(np.float32)
print(f"模拟特征图形状: {feature_maps.shape}")

# 方法1: 使用Keras内置层
keras_gap = keras.layers.GlobalAveragePooling2D()
gap_keras = keras_gap(feature_maps)

# 方法2: 使用tf.reduce_mean
gap_manual = tf.reduce_mean(feature_maps, axis=[1, 2])

# 方法3: 使用Lambda层封装
gap_lambda = keras.layers.Lambda(
    lambda X: tf.reduce_mean(X, axis=[1, 2])
)
gap_lambda_output = gap_lambda(feature_maps)

print(f"\nKeras GlobalAveragePooling2D 输出形状: {gap_keras.shape}")
print(f"tf.reduce_mean 输出形状: {gap_manual.shape}")
print(f"Lambda层 输出形状: {gap_lambda_output.shape}")

# 验证结果一致性
diff = np.max(np.abs(gap_keras.numpy() - gap_manual.numpy()))
print(f"\n结果差异: {diff:.2e} (应为0或极小)")

## 第四部分：在网络中应用深度池化

In [ ]:
def build_model_with_depth_pooling():
    """
    构建包含深度池化的CNN模型
    
    深度池化用于降低中间层的通道数，减少计算量
    """
    model = keras.Sequential([
        # 输入层
        keras.layers.InputLayer(input_shape=(28, 28, 1)),
        
        # 第一卷积块: 1 -> 32通道
        keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        keras.layers.MaxPooling2D(2),  # 空间池化: 28->14
        
        # 第二卷积块: 32 -> 64通道
        keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        # 深度池化: 64 -> 32通道
        keras.layers.Lambda(
            lambda x: tf.nn.max_pool(x, ksize=(1,1,1,2), strides=(1,1,1,2), padding='SAME')
        ),
        keras.layers.MaxPooling2D(2),  # 空间池化: 14->7
        
        # 第三卷积块
        keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        
        # 分类头
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    return model

# 构建并查看模型
model = build_model_with_depth_pooling()
model.summary()

In [ ]:
# 验证模型可以正常运行
(X_train, y_train), _ = keras.datasets.fashion_mnist.load_data()
X_train = X_train[:500].reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = y_train[:500]

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("验证模型训练...")
with tf.device('/CPU:0'):  # 深度池化需要在CPU上运行
    history = model.fit(X_train, y_train, epochs=2, batch_size=32, verbose=1)
    
print("模型训练验证完成")

## 总结

### 深度池化的特点

| 特性 | 说明 |
|-----|------|
| 操作维度 | 通道维度(C)而非空间维度(H,W) |
| 主要作用 | 降低特征图深度，融合相邻通道特征 |
| 硬件限制 | 目前仅支持CPU运算 |
| 实现方式 | `tf.nn.max_pool`配合特定ksize参数 |

### 使用场景

1. 需要在保持空间分辨率的同时降低通道数
2. 希望学习通道间的不变性表示
3. 特征融合和通道压缩

### 注意事项

- 深度池化目前仅支持CPU运算，可能影响训练速度
- 输入通道数应能被pool_size整除（或使用SAME填充）
- 实际应用中1x1卷积(瓶颈层)是更常用的通道降维方法